In [7]:
import sys
import numpy as np
import h5py
from functools import reduce
import operator

from scipy.special import logsumexp

sys.path.append('../')
from populations import *
from populations.bbh_models import get_models
from populations import bbh_models as read_models
from populations.utils.flow import NFlow
from populations.Flowsclass_dev import FlowModel
from populations import gw_obs

def test_alpha(chnl):
    #tests that alpha is calculated correctly for flow model for chnl
    #returns alpha calculated for flow model
    file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
    params = ['mchirp','q', 'chieff', 'z']
    device='cpu'
    detectable=True
    popsynth_outputs = read_models.read_hdf5(file_path, chnl)
    sensitivity ='midhighlatelow_network'

    if chnl=='CE':
        #CE channel
        alpha = np.zeros((4,5))

        for chib_id in range(4):
            for alphaCE_id in range(5):
                samples = popsynth_outputs[(chib_id,alphaCE_id)]
                mock_samp = samples.sample(int(1e6), weights=(samples['weight']/len(samples)), replace=True)
                alpha[chib_id,alphaCE_id] = np.sum(mock_samp['pdet_'+sensitivity]) / len(mock_samp)

        FlowPop = FlowModel.from_samples(chnl, popsynth_outputs, params, device='cpu', sensitivity=sensitivity, detectable=detectable)
        alpha_flow =FlowPop.alpha

        #reshape alpha_model into array from dict
        alpha_flow = np.reshape(list(alpha_flow.values()),(4,5))
    else:
        #non-CE channel
        alpha = np.zeros((4))

        for chib_id in range(4):
                samples = popsynth_outputs[(chib_id)]
                mock_samp = samples.sample(int(1e6), weights=(samples['weight']/len(samples)), replace=True)
                alpha[chib_id] = np.sum(mock_samp['pdet_'+sensitivity]) / len(mock_samp)

        FlowPop = FlowModel.from_samples(chnl, popsynth_outputs, params, device='cpu', sensitivity=sensitivity, detectable=detectable)
        alpha_flow =FlowPop.alpha

        #reshape alpha_model into array from dict
        alpha_flow = np.reshape(list(alpha_flow.values()),(4))

    #calculate difference
    alpha_difference=alpha-alpha_flow
    percent_difference=(alpha-alpha_flow)/alpha

    return alpha_flow

def test_alpha_KDEs(chnl):
    #tests that alpha is calculated correctly for KDE models for chnl= e.g. ['CE']
    #returns alpha calculated for each KDE model
    file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
    params = ['mchirp','q', 'chieff', 'z']
    device='cpu'
    detectable=True
    sensitivity ='midhighlatelow_network'
    popsynth_outputs = read_models.read_hdf5(file_path, chnl[0])
                
    model_names, KDEModels = get_models(file_path, chnl, params, use_flows=False,sensitivity=sensitivity, detectable=detectable)

    hyperparams = list(set([x.split('/', 1)[1] for x in model_names]))
    Nhyper = np.max([len(x.split('/')) for x in hyperparams])
    channels = sorted(list(set([x.split('/')[0] for x in model_names])))
    
    submodels_dict = {} #dummy index dict keys:0,1,2,3, items: particular models
    ctr=0 #associates with either chi_b or alpha (0 or 1)
    while ctr < Nhyper:
        submodels_dict[ctr] = {}
        hyper_set = sorted(list(set([x.split('/')[ctr] for x in hyperparams])))
        for idx, model in enumerate(hyper_set): #idx associates with 0,1,2,3,(4) keys
            submodels_dict[ctr][idx] = model
        ctr += 1


    if chnl==['CE']:
        alpha_kde = np.zeros((4,5))
        alpha = np.zeros((4,5))

        for chib_id in range(4):
            for alphaCE_id in range(5):
                samples = popsynth_outputs[(chib_id,alphaCE_id)]
                mock_samp = samples.sample(int(1e6), weights=(samples['weight']/len(samples)), replace=True)
                alpha[chib_id,alphaCE_id] = np.sum(mock_samp['pdet_'+sensitivity]) / len(mock_samp)

                model_list = []
                model_list.append(submodels_dict[0][chib_id])
                model_list.append(submodels_dict[1][alphaCE_id])
                model_list.insert(0,chnl[0])
                smdl = reduce(operator.getitem, model_list, KDEModels)
                alpha_kde[chib_id,alphaCE_id] = smdl.alpha
    else:
        alpha_kde = np.zeros((4))
        alpha = np.zeros((4))

        for chib_id in range(4):
                samples = popsynth_outputs[(chib_id)]
                mock_samp = samples.sample(int(1e6), weights=(samples['weight']/len(samples)), replace=True)
                alpha[chib_id] = np.sum(mock_samp['pdet_'+sensitivity]) / len(mock_samp)
                
                model_list = []
                model_list.append(submodels_dict[0][chib_id])
                model_list.insert(0,chnl[0])
                smdl = reduce(operator.getitem, model_list, KDEModels)
                alpha_kde[chib_id] = smdl.alpha

    return alpha_kde



In [10]:
for chnl in ['CE', 'CHE', 'GC', 'NSC', 'SMT']:
    flow_alpha=test_alpha(chnl)
    kde_alpha=test_alpha_KDEs([chnl])
    print((flow_alpha-kde_alpha)/flow_alpha)

100%|██████████| 20/20 [00:14<00:00,  1.38it/s]


[[-0.01406087 -0.04509856  0.00532175  0.03261551  0.00587357]
 [ 0.01949979  0.0317089   0.02506594 -0.02143708  0.00143551]
 [ 0.00275065 -0.02909689 -0.01111477 -0.00037518 -0.00773336]
 [ 0.00332116 -0.02373404 -0.01511633 -0.00204621 -0.0131591 ]]


100%|██████████| 4/4 [00:01<00:00,  2.22it/s]


[0.00376515 0.00218876 0.00457849 0.00235469]


100%|██████████| 4/4 [00:01<00:00,  2.35it/s]


[ 0.00217743 -0.00945051 -0.00170473  0.00180817]


100%|██████████| 4/4 [00:01<00:00,  2.60it/s]


[-0.0009398   0.00811158 -0.00462125  0.00529015]


100%|██████████| 4/4 [00:02<00:00,  1.63it/s]


[-0.01141815 -0.00290743  0.00325688 -0.0070149 ]
